# Item 71: Know How to Recognize When Concurrency Is Necessary




    a classic illustration of finite state automata. 
    The rules of the game are simple: 
    There’s a two-dimensional grid of an arbitrary size, and each cell in the grid can either be alive or empty:
        ALIVE = "*"
        EMPTY = "-"
    Based on its neighbor count, a cell decides if it will keep living, die, or regenerate.

In [5]:
import time 

ALIVE = "*"
EMPTY = "-"

class Grid:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def get(self, y, x):
        return self.rows[y % self.height][x % self.width]

    def set(self, y, x, state):
        self.rows[y % self.height][x % self.width] = state

    def __str__(self):
        output = []
        for row in self.rows:
            output.append(''.join(row))
        return '\n'.join(output)

grid = Grid(5, 9)
grid.set(0, 3, ALIVE)
grid.set(1, 4, ALIVE)
grid.set(2, 2, ALIVE)
grid.set(2, 3, ALIVE)
grid.set(2, 4, ALIVE)

# 以给定的坐标(x,y),计算及上、上右、右、右下等相邻节点的状态
def count_neighbors(y, x, get_cell):
    n_ = get_cell(y - 1, x + 0)  # North
    ne = get_cell(y - 1, x + 1)  # Northeast
    e_ = get_cell(y + 0, x + 1)  # East
    se = get_cell(y + 1, x + 1)  # Southeast
    s_ = get_cell(y + 1, x + 0)  # South
    sw = get_cell(y + 1, x - 1)  # Southwest
    w_ = get_cell(y + 0, x - 1)  # West
    nw = get_cell(y - 1, x - 1)  # Northwest
    neighbor_states = [n_, ne, e_, se, s_, sw, w_, nw]
    count = 0
    for state in neighbor_states:
        if state == ALIVE:
            count += 1
    return count

def game_logic(state, neighbors):
    time.sleep(0.1)
    if state == ALIVE:
        if neighbors < 2:
            return EMPTY
        elif neighbors > 3:
            return EMPTY
    else: 
        if neighbors == 3: 
            return ALIVE 
    return state

def step_cell(y, x, get_cell, set_cell):
    state = get_cell(y, x)
    neighbors = count_neighbors(y, x, get_cell)
    next_state = game_logic(state, neighbors)
    set_cell(y, x, next_state)

def simulate(grid):
    next_grid = Grid(grid.height, grid.width)
    for y in range(grid.height):
        for x in range(grid.width):
            step_cell(y, x, grid.get, next_grid.set)
    return next_grid

class ColumnPrinter:
    def __init__(self):
        self.columns = []
    
    def append(self, data):
        self.columns.append(data)
    
    def __str__(self):
        """将所有网格状态水平排列显示"""
        if not self.columns:
            return ""
        
        # 找出最大行数
        row_count = 1
        for data in self.columns:
            row_count = max(row_count, data.count('\n') + 1)
        
        # 初始化结果行
        rows = [''] * row_count
        
        # 处理每一列（每个网格状态）
        for j, data in enumerate(self.columns):
            lines = data.split('\n')
            
            # 处理当前列的每一行
            for i, line in enumerate(lines):
                if i < row_count:
                    rows[i] += line
                    
                    # 在列之间添加分隔符（除了最后一列）
                    if j < len(self.columns) - 1:
                        rows[i] += ' | '
            
            # 如果当前列的行数少于最大行数，用空格填充
            for i in range(len(lines), row_count):
                if j < len(self.columns) - 1:
                    rows[i] += ' ' * len(lines[0]) + ' | '
                else:
                    rows[i] += ' ' * len(lines[0])
        
        return '\n'.join(rows)

 
start = time.perf_counter()
columns = ColumnPrinter()
for i in range(5):
    columns.append(str(grid))
    grid = simulate(grid)

print(columns)
delta = time.perf_counter() - start
print('耗时: ', delta)



---*----- | --------- | --------- | --------- | ---------
----*---- | --*-*---- | ----*---- | ---*----- | ----*----
--***---- | ---**---- | --*-*---- | ----**--- | -----*---
--------- | ---*----- | ---**---- | ---**---- | ---***---
--------- | --------- | --------- | --------- | ---------
耗时:  23.280642000005173


# Things to Remember

- As a program’s scope and complexity increase, it often starts requiring support for multiple concurrent lines of execution.
- The most common types of concurrency coordination are fan-out (generating new units of concurrency) and fan-in (waiting for existing units of concurrency to complete).
- Python has many different ways of achieving fan-out and fan-in.